# COVID-19 analysis

## Summary
<img src="https://upload.wikimedia.org/wikipedia/commons/d/d5/Coronavirus_COVID-19_virus.jpg" style="width: 500px; padding: 5px; border: 1px solid #808080;">
    
> Coronavirus disease 2019 (COVID-19) is an infectious disease caused by the **severe acute respiratory syndrome coronavirus 2** (**SARS-CoV-2**) The disease has spread globally since 2019, resulting in the **2019–20 coronavirus pandemic**. Common symptoms include fever, cough and shortness of breath. Muscle pain, sputum production and sore throat are some of the less common symptoms. While the majority of cases result in mild symptoms, **some progress to pneumonia and multi-organ failure**. The case fatality rate is **estimated at between 1% and 5%** but varies by age and other health conditions.
> &mdash; <cite>[Wikipedia](https://en.wikipedia.org/wiki/Coronavirus_disease_2019)</cite>

## Datasets
* [Novel Corona Virus 2019 dataset](https://www.kaggle.com/sudalairajkumar/novel-corona-virus-2019-dataset): source [Johns Hopkins github](https://github.com/CSSEGISandData/COVID-19)
* [ECDC COVID-19 data](https://www.ecdc.europa.eu/en/publications-data/download-todays-data-geographic-distribution-covid-19-cases-worldwide)

## TODO:

* ~~Try some of the other available Kaggle datasets~~
* ~~Deal with provinces within countries (dataset heterogeneity)~~
* Calculate average time to recovery

## Further resources
* [Johns Hopkins map](https://coronavirus.jhu.edu/map.html)
* [Wikipedia COVID-19 page](https://en.wikipedia.org/wiki/Coronavirus_disease_2019)
* [NHS England COVID-19 advice for healthcare professionls](https://www.england.nhs.uk/coronavirus/)

### Module and data imports

In [ ]:
"""
Kaggle project: Analysis of COVID-19 CSV data
module and data imports
"""
import datetime
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

pd.plotting.register_matplotlib_converters()

%matplotlib inline
sns.set()
plt.rcParams['figure.figsize'] = (10,6)

data_dir = '/kaggle/input/novel-corona-virus-2019-dataset'

# relate the list indices (from the search above), to sensible hash table values
data_files = {
    'confirmed': 'time_series_covid_19_confirmed.csv',
    'recovered': 'time_series_covid_19_recovered.csv',
    'complete': 'covid_19_data.csv',
    'deaths': 'time_series_covid_19_deaths.csv',
}

DF = { key: pd.read_csv(os.path.join(data_dir, data_files[key])) for key in data_files}
DF['complete'] = DF['complete'].astype({'ObservationDate': 'Datetime64', 'Last Update': 'Datetime64'}).set_index('SNo') # cast
complete = DF['complete'] # shortcut (main dataset)
# some provinces are empty, others are filled with country/region -> ensure consistency
complete['Province/State'].fillna(complete['Country/Region'], inplace=True)
complete.set_index(['Country/Region', 'Province/State'], inplace=True)

### Function definitions

In [ ]:
def plot_country(country_name: str, log_scale: bool = False) -> list:
    """
    Plot a line chart of cases in a give country
    """
    country_cases = RAW_CASE_DATA['complete'].loc[country_name]
    # any regions with null values -> fill with the country name
    country_cases.index = country_cases.index.fillna(country_name)
    ax=list()
    ax.append(sns.lineplot(x='ObservationDate', y='Confirmed', data=country_cases))
    ax.append(sns.lineplot(x='ObservationDate', y='Deaths', data=country_cases))
    ax.append(sns.lineplot(x='ObservationDate', y='Recovered', data=country_cases))
    ax[0].axes.set_xlim(right=datetime.date.today())
    plt.suptitle('{} COVID-19 Cases'.format(country_name.title()))
    plt.xlabel('Date')
    plt.xticks(rotation=45)
    plt.ylabel('Cases')
    return ax


def country_summary(country_name: str):
    """
    Return key statistics for a specified countries
    """
    subframe = RAW_CASE_DATA['complete'].loc[country_name]
    return {'cases': subframe['Confirmed'].max(),
    'deaths': subframe['Deaths'].max(),
    'recoveries': subframe['Recovered'].max(),
    'prevalence': subframe['Confirmed'].max() - subframe['Recovered'].max() - subframe['Deaths'].max()
    }

In [ ]:
plot_country('Australia')
country_summary('Australia')